In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/rld/assignment5_materials/assignment5_materials'

/content/drive/MyDrive/rld/assignment5_materials/assignment5_materials


In [3]:
!ls -ltr

total 7585
-rw------- 1 root root     549 May  4 20:50  utils.py
-rw------- 1 root root     844 May  4 20:50  model.py
-rw------- 1 root root    2352 May  4 20:50  memory.py
-rw------- 1 root root    2919 May  4 20:50  agent.py
-rw------- 1 root root   20465 May  5 14:38  breakout_dqn.png
-rw------- 1 root root   21243 May  5 18:28  breakout_dqn_taaha_da_1.png
-rw------- 1 root root  468897 May  5 21:06 'Copy of MP5.ipynb'
-rw------- 1 root root   19595 May  5 21:06  breakout_dqn_n2.png
-rw------- 1 root root     376 May  5 22:17  config.py
-rw------- 1 root root  328299 May  6 02:27  Taaha_9pm_1.ipynb
-rw------- 1 root root   18751 May  6 02:27  breakout_dqn_Taaha_9pm.png
-rw------- 1 root root    7590 May  6 02:47 'CS 444 MP5 Report.docx'
-rw------- 1 root root   18239 May  6 03:07  breakout_dqn_n1.png
-rw------- 1 root root   44628 May  6 03:12  MP5.ipynb
drwx------ 2 root root    4096 May  6 05:13  __pycache__
drwx------ 2 root root    4096 May  6 05:13  old
-rw------- 1 root root 

# Deep Q-Learning 

Install dependencies for AI gym to run properly (shouldn't take more than a minute). If running on google cloud or running locally, only need to run once. Colab may require installing everytime the vm shuts down.

In [24]:
!pip3 install gym==0.25.2 pyvirtualdisplay
!sudo apt-get install -y xvfb python-opengl ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-2build1).
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
xvfb is already the newest version (2:1.20.13-1ubuntu1~20.04.8).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [5]:
!pip3 install --upgrade setuptools --user
!pip3 install ez_setup 
!pip3 install gym[atari] 
!pip3 install gym[accept-rom-license] 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ez_setup: filename=ez_setup-0.9-py3-none-any.whl size=11012 sha256=3b926d5073ca5ca2432bd25315c437258f4587875e9569f47d5691a9715d1cc8
  Stored in directory: /root/.cache/pip/wheels/7a/d6/77/8f495e85fb7df23d41c328b9ea3cf0d9e83631b20bba479293
Successfully built ez_setup
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 9.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproj

For this assignment we will implement the Deep Q-Learning algorithm with Experience Replay as described in breakthrough paper __"Playing Atari with Deep Reinforcement Learning"__. We will train an agent to play the famous game of __Breakout__.

In [6]:
%matplotlib inline

import sys
import gym
import torch
import pylab
import random
import numpy as np
from collections import deque
from datetime import datetime
from copy import deepcopy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from utils import find_max_lives, check_live, get_frame, get_init_state
from model import DQN
from config import *

import matplotlib.pyplot as plt
# %load_ext autoreload
# %autoreload 2

## Understanding the environment

In the following cell, we initialize our game of __Breakout__ and you can see how the environment looks like. For further documentation of the of the environment refer to https://www.gymlibrary.dev/environments/atari/breakout/. 

In breakout, we will use 3 actions "fire", "left", and "right". "fire" is only used to reset the game when a life is lost, "left" moves the agent left and "right" moves the agent right.

In [7]:
env = gym.make('BreakoutDeterministic-v4')
state = env.reset()

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [8]:
number_lives = find_max_lives(env)
state_size = env.observation_space.shape
action_size = 3 #fire, left, and right

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


## Creating a DQN Agent

Here we create a DQN Agent. This agent is defined in the __agent.py__. The corresponding neural network is defined in the __model.py__. Once you've created a working DQN agent, use the code in agent.py to create a double DQN agent in __agent_double.py__. Set the flag "double_dqn" to True to train the double DQN agent.

__Evaluation Reward__ : The average reward received in the past 100 episodes/games.

__Frame__ : Number of frames processed in total.

__Memory Size__ : The current size of the replay memory.

In [9]:
#double_dqn = False # set to True if using double DQN agent
double_dqn = True
if double_dqn:
    from agent_double import Agent
else:
    from agent import Agent

agent = Agent(action_size)
evaluation_reward = deque(maxlen=evaluation_reward_length)
frame = 0
memory_size = 0

In [11]:
!cp /content/drive/MyDrive/rld/assignment5_materials/assignment5_materials/1_am_breakout_dqn.pth /content/

In [15]:
agent.load_policy_net('/content/1_am_breakout_dqn.pth')

### Main Training Loop

In this training loop, we do not render the screen because it slows down training signficantly. To watch the agent play the game, run the code in next section "Visualize Agent Performance"

In [ ]:
rewards, episodes = [], []
best_eval_reward = 0
for e in range(EPISODES):
    done = False
    score = 0

    history = np.zeros([5, 84, 84], dtype=np.uint8)
    step = 0
    state = env.reset()
    next_state = state
    life = number_lives

    get_init_state(history, state, HISTORY_SIZE)

    while not done:
        step += 1
        frame += 1

        # Perform a fire action if ball is no longer on screen to continue onto next life
        if step > 1 and len(np.unique(next_state[:189] == state[:189])) < 2:
            action = 0
        else:
            action = agent.get_action(np.float32(history[:4, :, :]) / 255.)
        state = next_state
        next_state, reward, done, info = env.step(action + 1)
        
        frame_next_state = get_frame(next_state)
        history[4, :, :] = frame_next_state
        terminal_state = check_live(life, info['lives'])

        life = info['lives']
        r = reward

        # Store the transition in memory 
        agent.memory.push(deepcopy(frame_next_state), action, r, terminal_state)
        # Start training after random sample generation
        
        if(frame >= train_frame):
            agent.train_policy_net(frame)
            # Update the target network only for Double DQN only
            if double_dqn and (frame % update_target_network_frequency)== 0:
                agent.update_target_net()
        score += reward
        history[:4, :, :] = history[1:, :, :]
            
        if done:
            evaluation_reward.append(score)
            rewards.append(np.mean(evaluation_reward))
            episodes.append(e)
            pylab.plot(episodes, rewards, 'b')
            pylab.xlabel('Episodes')
            pylab.ylabel('Rewards') 
            pylab.title('Episodes vs Reward')
            pylab.savefig("1_am.png") # save graph for training visualization
            
            # every episode, plot the play time
            print("episode:", e, "  score:", score, "  memory length:",
                  len(agent.memory), "  epsilon:", agent.epsilon, "   steps:", step,
                  "   lr:", agent.optimizer.param_groups[0]['lr'], "    evaluation reward:", np.mean(evaluation_reward))

            # if the mean of scores of last 100 episode is bigger than 5 save model
            ### Change this save condition to whatever you prefer ###
            if np.mean(evaluation_reward) > 5 and np.mean(evaluation_reward) > best_eval_reward:
                torch.save(agent.policy_net, "1_am_breakout_dqn.pth")
                best_eval_reward = np.mean(evaluation_reward)


episode: 0   score: 2.0   memory length: 219   epsilon: 1.0    steps: 219    lr: 0.0001     evaluation reward: 2.0
episode: 1   score: 0.0   memory length: 343   epsilon: 1.0    steps: 124    lr: 0.0001     evaluation reward: 1.0
episode: 2   score: 0.0   memory length: 466   epsilon: 1.0    steps: 123    lr: 0.0001     evaluation reward: 0.6666666666666666
episode: 3   score: 2.0   memory length: 686   epsilon: 1.0    steps: 220    lr: 0.0001     evaluation reward: 1.0
episode: 4   score: 2.0   memory length: 887   epsilon: 1.0    steps: 201    lr: 0.0001     evaluation reward: 1.2
episode: 5   score: 0.0   memory length: 1011   epsilon: 1.0    steps: 124    lr: 0.0001     evaluation reward: 1.0
episode: 6   score: 2.0   memory length: 1231   epsilon: 1.0    steps: 220    lr: 0.0001     evaluation reward: 1.1428571428571428
episode: 7   score: 3.0   memory length: 1499   epsilon: 1.0    steps: 268    lr: 0.0001     evaluation reward: 1.375
episode: 8   score: 2.0   memory length: 1698

/content/drive/MyDrive/rld/assignment5_materials/assignment5_materials/memory.py:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sample = np.array(sample)
/content/drive/MyDrive/rld/assignment5_materials/assignment5_materials/agent_double.py:80: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mini_batch = np.array(mini_batch).transpose()


episode: 541   score: 3.0   memory length: 100143   epsilon: 0.9997148800000062    steps: 230    lr: 0.0001     evaluation reward: 1.69
episode: 542   score: 2.0   memory length: 100341   epsilon: 0.9993228400000147    steps: 198    lr: 0.0001     evaluation reward: 1.69
episode: 543   score: 0.0   memory length: 100465   epsilon: 0.99907732000002    steps: 124    lr: 0.0001     evaluation reward: 1.67
episode: 544   score: 2.0   memory length: 100684   epsilon: 0.9986437000000294    steps: 219    lr: 0.0001     evaluation reward: 1.65
episode: 545   score: 1.0   memory length: 100836   epsilon: 0.998342740000036    steps: 152    lr: 0.0001     evaluation reward: 1.64
episode: 546   score: 0.0   memory length: 100960   epsilon: 0.9980972200000413    steps: 124    lr: 0.0001     evaluation reward: 1.62
episode: 547   score: 3.0   memory length: 101207   epsilon: 0.9976081600000519    steps: 247    lr: 0.0001     evaluation reward: 1.64
episode: 548   score: 2.0   memory length: 101408  

/content/drive/MyDrive/rld/assignment5_materials/assignment5_materials/memory.py:30: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  sample = np.array(sample)


episode: 554   score: 0.0   memory length: 102481   epsilon: 0.9950856400001067    steps: 124    lr: 0.0001     evaluation reward: 1.64
episode: 555   score: 2.0   memory length: 102680   epsilon: 0.9946916200001152    steps: 199    lr: 0.0001     evaluation reward: 1.66
episode: 556   score: 0.0   memory length: 102804   epsilon: 0.9944461000001206    steps: 124    lr: 0.0001     evaluation reward: 1.65
episode: 557   score: 1.0   memory length: 102974   epsilon: 0.9941095000001279    steps: 170    lr: 0.0001     evaluation reward: 1.66
episode: 558   score: 2.0   memory length: 103173   epsilon: 0.9937154800001364    steps: 199    lr: 0.0001     evaluation reward: 1.68
episode: 559   score: 0.0   memory length: 103296   epsilon: 0.9934719400001417    steps: 123    lr: 0.0001     evaluation reward: 1.67
episode: 560   score: 2.0   memory length: 103494   epsilon: 0.9930799000001502    steps: 198    lr: 0.0001     evaluation reward: 1.68
episode: 561   score: 5.0   memory length: 10382

/content/drive/MyDrive/rld/assignment5_materials/assignment5_materials/agent_double.py:80: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  mini_batch = np.array(mini_batch).transpose()


episode: 562   score: 3.0   memory length: 104093   epsilon: 0.991893880000176    steps: 267    lr: 0.0001     evaluation reward: 1.69
episode: 563   score: 3.0   memory length: 104328   epsilon: 0.9914285800001861    steps: 235    lr: 0.0001     evaluation reward: 1.63
episode: 564   score: 2.0   memory length: 104527   epsilon: 0.9910345600001946    steps: 199    lr: 0.0001     evaluation reward: 1.63
episode: 565   score: 2.0   memory length: 104726   epsilon: 0.9906405400002032    steps: 199    lr: 0.0001     evaluation reward: 1.62
episode: 566   score: 0.0   memory length: 104850   epsilon: 0.9903950200002085    steps: 124    lr: 0.0001     evaluation reward: 1.6
episode: 567   score: 3.0   memory length: 105098   epsilon: 0.9899039800002192    steps: 248    lr: 0.0001     evaluation reward: 1.62
episode: 568   score: 2.0   memory length: 105296   epsilon: 0.9895119400002277    steps: 198    lr: 0.0001     evaluation reward: 1.62
episode: 569   score: 0.0   memory length: 105420 

# Visualize Agent Performance

BE AWARE THIS CODE BELOW MAY CRASH THE KERNEL IF YOU RUN THE SAME CELL TWICE.

Please save your model before running this portion of the code.

In [20]:
#from gym.wrappers import Monitor # If importing monitor raises issues, try using `from gym.wrappers import RecordVideo`
from gym.wrappers.record_video import RecordVideo

import glob
import io
import base64

from IPython.display import HTML
from IPython import display as ipythondisplay

from pyvirtualdisplay import Display

# Displaying the game live
def show_state(env, step=0, info=""):
    plt.figure(3)
    plt.clf()
    plt.imshow(env.render(mode='rgb_array'))
    plt.title("%s | Step: %d %s" % ("Agent Playing",step, info))
    plt.axis('off')

    ipythondisplay.clear_output(wait=True)
    ipythondisplay.display(plt.gcf())
    
# Recording the game and replaying the game afterwards
def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")
    

def wrap_env(env):
    env = RecordVideo(env, './video')
    return env

In [28]:
display = Display(visible=0, size=(300, 200))
display.start()

# Load agent
# agent.load_policy_net("./save_model/breakout_dqn.pth")
agent.epsilon = 0.0 # Set agent to only exploit the best action

env = gym.make('BreakoutDeterministic-v4')
env = wrap_env(env)

done = False
score = 0
step = 0
state = env.reset()
next_state = state
life = number_lives
history = np.zeros([5, 84, 84], dtype=np.uint8)
get_init_state(history, state, history.shape[0])

while not done:
    
    # Render breakout
    env.render()
#     show_state(env,step) # uncommenting this provides another way to visualize the game

    step += 1
    frame += 1

    # Perform a fire action if ball is no longer on screen
    if step > 1 and len(np.unique(next_state[:189] == state[:189])) < 2:
        action = 0
    else:
        action = agent.get_action(np.float32(history[:4, :, :]) / 255.)
    state = next_state
    
    next_state, reward, done, info = env.step(action + 1)
        
    frame_next_state = get_frame(next_state)
    history[4, :, :] = frame_next_state
    #print(info)
    terminal_state = check_live(life, info['lives'])
        
    life = info['lives']
    r = np.clip(reward, -1, 1) 
    r = reward

    # Store the transition in memory 
    agent.memory.push(deepcopy(frame_next_state), action, r, terminal_state)
    # Start training after random sample generation
    score += reward
    
    history[:4, :, :] = history[1:, :, :]
env.close()
show_video()
display.stop()

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/record_video.py:78: UserWarning: WARN: Overwriting existing videos at /content/drive/MyDrive/rld/assignment5_materials/assignment5_materials/video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.